In [54]:
%load_ext autoreload
%autoreload 2
import os
import matplotlib.pyplot as plt
import seaborn as sns
from os.path import join
from tqdm import tqdm
import pandas as pd
import imodelsx.process_results
import sys
import datasets
import numpy as np
from copy import deepcopy
import clin.eval
import clin.modules.prune
import clin.modules.evidence
import clin.modules.omission
import clin.modules.status
import clin.llm
import clin.parse
from collections import defaultdict
import openai
from typing import List, Tuple
openai.api_key_path = '/home/chansingh/.OPENAI_KEY'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [55]:
# download and extract data
# !wget https://github.com/bepnye/EBM-NLP/raw/master/ebm_nlp_2_00.tar.gz
# !tar -xvf ebm_nlp_2_00.tar.gz

In [56]:
import os
import os.path
from os.path import join

In [57]:
DATA_DIR = 'ebm_nlp_2_00'
DOC_DIR = join(DATA_DIR, 'documents')
ANNOT_DIR = join(DATA_DIR, 'annotations', 'aggregated', 'starting_spans', 'interventions', 'test', 'gold')

In [58]:
annot_fnames = os.listdir(ANNOT_DIR)
doc_ids_gold = [fname.split('.')[0] for fname in annot_fnames]
doc_names = sorted(os.listdir(join(DATA_DIR, 'documents')))
doc_ids = list(set([doc_name.split('.')[0] for doc_name in doc_names]))
# find doc_ids that are in doc_ids_gold
doc_ids = [doc_id for doc_id in doc_ids if doc_id in doc_ids_gold]

In [59]:
i = 0
doc = open(join(DOC_DIR, doc_ids[i] + '.txt'), 'r').read()
toks = open(join(DOC_DIR, doc_ids[i] + '.tokens'), 'r').read()
annot = open(join(ANNOT_DIR, doc_ids[i] + '.AGGREGATED.ann'), 'r').read()

In [61]:
print(doc)
toks_list = toks.split()
annot_list = np.array([int(i) for i in annot.split()]).astype(int)
print(annot_list)
annot_arr = (annot_list > 0).astype(int)
print(annot_arr)

def find_continguous_sequences(annot_arr: np.ndarray) -> List[Tuple]:
    # return a list of (start, stop) tuples identifying continguous sequences where annot_list is 1
    # e.g. annot_arr = [0, 1, 1, 0, 1, 1, 1, 0, 0, 1] -> [(1, 3), (4, 7), (9, 11)]
    annot_arr = np.concatenate([[0], annot_arr, [0]])
    diffs = np.diff(annot_arr)
    starts = np.where(diffs == 1)[0]
    stops = np.where(diffs == -1)[0]
    return list(zip(starts, stops))

contiguous_seqs = find_continguous_sequences(annot_arr)
for start, stop in contiguous_seqs:
    print(toks_list[start:stop])

Effects of mild physical exercise on serum lipoproteins and metabolites of arachidonic acid: a controlled randomised trial in middle aged men.

To study the effects of physical exercise on biochemical risk factors for ischaemic heart disease 31 healthy middle aged men undertook regular physical exercise for two months and 29 served as controls in a randomised trial. In the men taking regular exercise serum cholesterol concentrations increased 26% more in the high density lipoprotein subfraction two (HDL2) and decreased 31% more in the subfraction three (HDL3) and 9% more in the low density lipoprotein fraction than in the control group. A tendency towards increased plasma 6-keto-prostaglandin F1 alpha concentration and decreased serum thromboxane B2 concentration was found during the period of regular exercise, but prostaglandin E2 concentrations remained unchanged. The increase in plasma 6-keto-prostaglandin F1 alpha concentration was associated with an increase in serum HDL2 choleste

In [46]:
annot_arr

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [36]:
len(annot_list)

199